In [ ]:
from datetime import datetime, timedelta
import pandas as pd

loans = pd.read_pickle('~/chtl-data/processed/loans.pkl')
items = pd.read_pickle('/home/max/chtl-data/processed/inventory.pkl')
items = items.set_index("Item ID")
joined = loans.join(items, on="Item ID", lsuffix="-l")

# I scanned a CSV of in maintenance items, and noted any that had Item Types I think should be grouped, or had a bad Item Type for a given Item ID.
item_type_overrides = {
    "Cordless Drills": "Power Drills", "Drills": "Power Drills", "Power Drill 3/8in": "Power Drills",
    "Flat Screwdrivers": "Screwdrivers",
    "Finishing Power Sanders": "Sanders"
}
for k, v in item_type_overrides.items():
    joined.loc[joined["Item Type"] == k, "Item Type"] = v
    items.loc[items["Item Type"] == k, "Item Type"] = v
    
item_id_type_overrides = { 3894: "Automotive Tools"}
for k, v in item_id_type_overrides.items():
    joined.loc[joined["Item ID"] == k, "Item Type"] = v
    items.loc[k, "Item Type"] = v

# Exclude any status that disables an item  from the maintenance list. This isn't perfect (a handful of items are incorrectly labeled In Maintenance and Disabled
# w/ the intention of fixing them), but correctly excludes a lot of things we've given up on.
in_maintenance = items[items['In Maintenance'] & ~(items['Disabled'] | items['Not Fixable'] | items['Lost In Shop'])]

def season(dt):
    if dt.month in (12, 1, 2):
        return "Winter"
    if dt.month in (3, 4, 5):
        return "Spring"
    if dt.month in (6, 7, 8):
        return "Summer"
    return "Fall"

joined["Season"] = joined["Checked Out"].map(season)
item_type_popularity = joined.groupby(["Item Type", "Season"], observed=False).size().reset_index().rename(columns={0: "Item Type Loan Count"})
popularity_by_season = item_type_popularity.pivot_table("Item Type Loan Count", ["Item Type"], "Season", observed=False)
popularity_by_season["Item Type Loan Count"] = popularity_by_season["Fall"] + popularity_by_season["Spring"] + popularity_by_season["Summer"] + popularity_by_season["Winter"]

# Exclude anything more than 6 weeks overdue from our "Item Type Available Count".
very_overdue = loans[(loans["Due Date"] < (datetime.now() - timedelta(weeks=6))) & pd.isna(loans['Checked In'])]
very_overdue = very_overdue.set_index("Item ID")
enabled_items = items[~(items["Disabled"] | items["Shop Use Only"] | items["Lost By Member"] | items["Lost In Shop"] | items["Not Fixable"] | items["In Maintenance"])]
items_w_overdue_info = enabled_items.join(very_overdue, rsuffix="-overdue")
item_type_count = items_w_overdue_info[items_w_overdue_info['index'].isna()].groupby(["Item Type"], observed=False).size()

with_popularity = in_maintenance.join(popularity_by_season, on="Item Type").join(item_type_count.rename("Item Type Available Item Count"), on="Item Type")
# TODO: Consider changing any items with 0 available items, to have a Usage Ratio that is just "# of times loaned", so they're not shot to the top of the list.
with_popularity["Usage Ratio"] = (with_popularity["Item Type Loan Count"] / with_popularity["Item Type Available Item Count"]).round(0)
today = datetime.today()
with_popularity["Current Season Usage Ratio"] = (with_popularity[season(today)] / with_popularity["Item Type Available Item Count"]).round(0)
# 13 weeks should be another season
next_season_date = today + timedelta(weeks=13)
with_popularity["Next Season Usage Ratio"] = (with_popularity[season(next_season_date)] / with_popularity["Item Type Available Item Count"]).round(0)
with_popularity.sort_values(by="Usage Ratio", ascending=False)[
  ["Item Type", "Name", "Item Type Loan Count", "Item Type Available Item Count", "Usage Ratio", "Current Season Usage Ratio", "Next Season Usage Ratio"]
].to_csv("~/chtl-data/reports/maintenance-by-usage-ratio.csv")
